In [3]:
# import pandas as pd

# # Load the dataset
# file_path = 'weatherHistory.csv'  # Pastikan ini adalah jalur file yang benar
# df = pd.read_csv(file_path)

# # Convert 'Formatted Date' to datetime
# df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])

# # Sort data by 'Formatted Date' to ensure it's in chronological order
# df = df.sort_values(by='Formatted Date')

# # Set 10% of the latest data as test data
# test_size = 0.1
# n_test = int(len(df) * test_size)

# # Split the data into train and test sets
# train_data = df.iloc[:-n_test]
# test_data = df.iloc[-n_test:]

# # Check the shape of train and test sets
# print(f"Train data shape: {train_data.shape}")
# print(f"Test data shape: {test_data.shape}")

# # Save the train and test data to separate CSV files
# train_data.to_csv('train_data.csv', index=False)
# test_data.to_csv('test_data.csv', index=False)


/var/folders/2c/6m6pkt056t9624rf6blc64kw0000gn/T/ipykernel_28244/3961996355.py:8: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['Formatted Date'] = pd.to_datetime(df['Formatted Date'])


Train data shape: (86808, 12)
Test data shape: (9645, 12)


In [4]:
# Load the dataset
file_path = 'weatherHistory.csv'  # Pastikan jalur ini sesuai dengan lokasi file Anda
df = pd.read_csv(file_path)

# Menampilkan kategori unik di kolom 'Summary' dan 'Precip Type'
summary_categories = df['Summary'].unique()
precip_type_categories = df['Precip Type'].unique()

print("Kategori unik di 'Summary':")
print(summary_categories)

print("\nKategori unik di 'Precip Type':")
print(precip_type_categories)


Kategori unik di 'Summary':
['Partly Cloudy' 'Mostly Cloudy' 'Overcast' 'Foggy'
 'Breezy and Mostly Cloudy' 'Clear' 'Breezy and Partly Cloudy'
 'Breezy and Overcast' 'Humid and Mostly Cloudy' 'Humid and Partly Cloudy'
 'Windy and Foggy' 'Windy and Overcast' 'Breezy and Foggy'
 'Windy and Partly Cloudy' 'Breezy' 'Dry and Partly Cloudy'
 'Windy and Mostly Cloudy' 'Dangerously Windy and Partly Cloudy' 'Dry'
 'Windy' 'Humid and Overcast' 'Light Rain' 'Drizzle' 'Windy and Dry'
 'Dry and Mostly Cloudy' 'Breezy and Dry' 'Rain']

Kategori unik di 'Precip Type':
['rain' 'snow' nan]


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix


In [11]:
# Membaca data training
data = pd.read_csv('train_data.csv')

# Membaca data testing
test_data = pd.read_csv('test_data.csv')


In [12]:
# Mengonversi kolom 'Formatted Date' ke UTC
data['Formatted Date'] = pd.to_datetime(data['Formatted Date'], utc=True)

# Atau menghapus zona waktu
data['Formatted Date'] = pd.to_datetime(data['Formatted Date']).dt.tz_localize(None)

# Mengekstrak fitur waktu
data['Year'] = data['Formatted Date'].dt.year
data['Month'] = data['Formatted Date'].dt.month
data['Day'] = data['Formatted Date'].dt.day
data['Hour'] = data['Formatted Date'].dt.hour


In [16]:
# Pastikan untuk menggunakan encoder yang sama dengan yang dipakai di training data
test_data['Summary'] = test_data['Summary'].apply(lambda x: x if x in label_encoder.classes_ else 'Unknown')

# Tambahkan label 'Unknown' pada label encoder
label_encoder.classes_ = np.append(label_encoder.classes_, 'Unknown')

# Transformasi data test
test_data['Summary'] = label_encoder.transform(test_data['Summary'])


AttributeError: 'LabelEncoder' object has no attribute 'classes_'

In [11]:
# Fitur (X) dan Label (y)
X_train = data.drop('Summary', axis=1)
y_train = data['Summary']

X_test = test_data.drop('Summary', axis=1)
y_test = test_data['Summary']

In [15]:
from sklearn.preprocessing import LabelEncoder

# Label encode 'Precip Type'
label_encoder = LabelEncoder()

# Pastikan semua kolom kategorikal sudah diencode
data['Precip Type'] = label_encoder.fit_transform(data['Precip Type'])
test_data['Precip Type'] = label_encoder.transform(test_data['Precip Type'])


In [18]:
# Menghapus kolom datetime jika ada
X_train = X_train.drop(['Formatted Date'], axis=1, errors='ignore')
X_test = X_test.drop(['Formatted Date'], axis=1, errors='ignore')


In [22]:
# One-hot encoding Daily Summary
X_train = pd.get_dummies(X_train, columns=['Daily Summary'])
X_test = pd.get_dummies(X_test, columns=['Daily Summary'])


In [9]:

# Melatih model setelah pembersihan
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


NameError: name 'X_train' is not defined